# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [1]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [2]:
# Create a temporary workspace
import tempfile


SESSION_WORKSPACE = tempfile.mkdtemp()
print(f"Session workspace created at: {SESSION_WORKSPACE}")

Session workspace created at: /tmp/tmpm1vock2i


In [3]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/My Drive"

Mounted at /content/drive


In [4]:
# Unzip the dataset
import shutil
import os


DATASET_DIR_NAME = "duckietown_object_detection_dataset"
DATASET_ZIP_NAME = f"{DATASET_DIR_NAME}.zip"
DATASET_DIR_PATH = os.path.join(SESSION_WORKSPACE, DATASET_DIR_NAME)
TRAIN_DIR = "train"
VALIDATION_DIR = "val"
IMAGES_DIR = "images"
LABELS_DIR = "labels"


def show_info(base_path: str):
  for l1 in [TRAIN_DIR, VALIDATION_DIR]:
    for l2 in [IMAGES_DIR, LABELS_DIR]:
      p = os.path.join(base_path, l1, l2)
      print(f"#Files in {l1}/{l2}: {len(os.listdir(p))}")


def unzip_dataset():
  # check zipped file
  zip_path = os.path.join(DRIVE_PATH, DATASET_ZIP_NAME)
  assert os.path.exists(zip_path), f"No zipped dataset found at {zip_path}! Abort!"

  # unzip the data
  print("Unpacking zipped data...")
  shutil.unpack_archive(zip_path, DATASET_DIR_PATH)
  print(f"Zipped dataset unpacked to {DATASET_DIR_PATH}")

  # show some info
  show_info(DATASET_DIR_PATH)


unzip_dataset()

Unpacking zipped data...
Zipped dataset unpacked to /tmp/tmpm1vock2i/duckietown_object_detection_dataset
#Files in train/images: 964
#Files in train/labels: 964
#Files in val/images: 242
#Files in val/labels: 242


In [5]:
# change  working directory to the session workspace
os.chdir(SESSION_WORKSPACE)
print(f"PWD: {os.getcwd()}")

# install pytorch and torchvision
!pip3 install torch==1.13.0 torchvision==0.14.0

PWD: /tmp/tmpm1vock2i
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 705.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu121
    Uninstalling torchvision-0.16.0+cu121:
      Successfully uninstalled torchvision-0.16.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

# Next, we will clone Yolov5

In [6]:
!rm -rf ./yolov5
!git clone https://github.com/ultralytics/yolov5.git -b v7.0
!cd yolov5 && pip3 install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16117, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 16117 (delta 9), reused 11 (delta 1), pack-reused 16089
Receiving objects: 100% (16117/16117), 14.86 MiB | 21.02 MiB/s, done.
Resolving deltas: 100% (11032/11032), done.
Note: switching to '915bbf294bb74c859f0b41f1c23bc395014ea679'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 1.7 MB/s eta 

# We now inform the training process of the format and location of our dataset

In [7]:
%%writefile ./yolov5/data/duckietown.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../duckietown_object_detection_dataset/train
val: ../duckietown_object_detection_dataset/val

# number of classes
nc: 4

# class names
names: [ 'duckie', 'cone', 'truck', 'bus' ]

Writing ./yolov5/data/duckietown.yaml


# And we're ready to train! This step will take about 5 minutes.

Notice that we're only training for 10 epochs. That's probably not enough!

In [8]:
!cd yolov5 && python3 train.py --cfg ./models/yolov5n.yaml --img 416 --batch 32 --epochs 10 --data duckietown.yaml --weights yolov5n.pt

train: weights=yolov5n.pt, cfg=./models/yolov5n.yaml, data=duckietown.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 254 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v7.0-0-g915bbf2 Python-3.10.12 torch-1.13.0+cu117 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0

In [9]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")

Latest exp is exp
None
Marked the model from the latest run (exp) as yolov5/best.pt.


# Next, we can upload your model to Duckietown's cloud!

We will need our token to access our personal cloud space.

In [10]:
# TODO: Fill in the duckietown token here
YOUR_DT_TOKEN = "dt1-3nT7FDbT7NLPrXykNJW6pwvofokYQzZeC8eNrAXJqeVCqtQ-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfg7g62sx5b32fiNw5wpSsqhKz3pMos4AJV"

Then, we chose the location of the trained model on disk and its name once uploaded to our cloud space. You should not change these values, or the robots will not be able to find the model to download.

In [11]:
import sys
sys.path.insert(0, './yolov5')

# DO NOT CHANGE THESE
model_name = "yolov5n"
model_local_path = "./yolov5/best.pt"
model_remote_path = f"courses/mooc/objdet/data/nn_models/{model_name}.pt"

# install DCSS client
!pip3 install dt-data-api-daffy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 1.6 MB/s eta 0:00:00
  Created wheel for dt-data-api-daffy: filename=dt_data_api_daffy-1.2.0-py3-none-any.whl size=14284 sha256=b890db9a7085181501a989f027d141852c2d262226137500402f5190a814acd8
  Stored in directory: /root/.cache/pip/wheels/74/53/2a/80f68075691d924b254fc10c2fd5de630705daadab01c961b4
  Created wheel for dt-authentication-daffy: filename=dt_authentication_daffy-0.1.16-py3-none-any.whl size=3140 sha256=78f6b016867689eb36e98da412858bd438584c0404cc3e55602f9cb86dd319e2
  Stored in directory: /root/.cache/pip/wheels/a1/02/64/f1edbb576157ed7ec13aa57e824ce80df9ab24080f3a0d8e37
Successfully built dt-data-api-daffy dt-authentication-daffy


We now open a pointer to our cloud space and upload the model.

In [12]:
import torch
from dt_data_api import DataClient, Storage

# open a pointer to our personal duckietown cloud space
client = DataClient(YOUR_DT_TOKEN)
storage = client.storage("user")

# upload model
upload = storage.upload(model_local_path, model_remote_path)
upload.join()

# Done!

We're done training! You can now close this tab and go back to the `Training` notebook